In [1]:
import findspark
findspark.init()

import pyspark
import random
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("LOL").getOrCreate()
sc = spark.sparkContext
import pyspark.sql.functions as f
from pyspark.sql.functions import col , explode , concat , expr , rank
from pyspark.sql import Window


In [3]:
#######################   Data loading ##################
path = "champion_class.json"
champion_class_dict = spark.read.json(path ,  multiLine=True)
path = "1.json"
matches = spark.read.json(path ,  multiLine=True)

In [4]:
######################################## Champions ###########################################

#######################################  Ban times ###########################################

Teams = matches.rdd.flatMap(lambda row: row['teams'])
Banned_champions = Teams.flatMap(lambda row: row['bans'])
Ban_times = Banned_champions.map(lambda row: (row['championId'] , 1)).reduceByKey(lambda a,b: a+b)

###################################### Pick times ############################################

participants = matches.rdd.flatMap(lambda row: row['participants'])
Pick_times = participants.map(lambda row: (row['championId'] ,1 )).reduceByKey(lambda a,b: a+b)

###################################### Win rate  #############################################

Winner_champions = participants.filter(lambda row: row['stats']['win']==True )
Champions_winning_time = Winner_champions.map(lambda row: (row['championId'] , 1)).reduceByKey(lambda a,b: a+b)
Winning_champions= Champions_winning_time.join(Pick_times)
champions_winning_rate = Winning_champions.map(lambda row : (row[0],row[1][0]/row[1][1]) )

##################################### Synergies  & Duos #####################################

Participants_in_Game = matches.select(explode("participants").alias("participants") , col("gameId").alias("gameId"))
Participants_in_Game_champions=Participants_in_Game.select(concat(col("gameId"),col("participants.teamId")).alias("Fullinfo"),"participants.championId")

dummy_right = Participants_in_Game_champions.withColumnRenamed("championId","right_championId").withColumnRenamed("Fullinfo","right_Fullinfo")
dummy_left = Participants_in_Game_champions.withColumnRenamed("championId","left_championId").withColumnRenamed("Fullinfo","left_Fullinfo")
p_champions_join = dummy_left.join(dummy_right , dummy_right.right_Fullinfo == dummy_left.left_Fullinfo , how = "outer" ).filter(dummy_right.right_championId != dummy_left.left_championId ).drop('left_Fullinfo').drop('right_Fullinfo')

synergies = p_champions_join.rdd.map(lambda row : ((row[0] , row[1] ) ,1)).reduceByKey(lambda a , b : a+b)
most_synergies = synergies.sortBy(lambda row : -row[1]).map(lambda row : (row[0][0] , row[0][1] ,row[1]))

## 
##################################### Duos Win rate ##########################################



In [5]:
######################################## Data preperation #####################################
######################################## Table of items #######################################


items_table = matches.select(explode("participants").alias("participants") ,col("gameId").alias("gameId") )\
                  .select(col("participants.stats.item0.name").alias("item0") ,
                          col("participants.stats.item1.name").alias("item1") ,
                          col("participants.stats.item2.name").alias("item2") ,
                          col("participants.stats.item3.name").alias("item3") ,
                          col("participants.stats.item4.name").alias("item4") ,
                          col("participants.stats.item5.name").alias("item5") ,
                          col("participants.stats.item6.name").alias("item6") ,
                          concat(col("participants.championId") ,col("gameId")).alias("Info") ,
                          col("participants.championId").alias("championId") ,
                          col("participants.stats.win").alias("win")  ,
                          )
### Stacking all the item in one coloum to ease the mapping process 
unpivotExpr = "stack(7, item0 , Info,  item1 , Info , item2 , Info  , item3 , Info ,item4, Info , item5 ,  Info , item6  , Info ) as (item , Info)"
Stacked_items_per_cahmpion = items_table.select("championId", expr(unpivotExpr) , "win")

### Mapping from champion to class
### Stacking all the item in one coloum to ease the mapping process 
items_per_champion_per_class = Stacked_items_per_cahmpion.join(champion_class_dict , Stacked_items_per_cahmpion.championId == champion_class_dict.championId , how = "left" ).drop("championId")
items_per_class  = items_per_champion_per_class.select( items_per_champion_per_class.item  , items_per_champion_per_class.tags[0].alias("tag1") , items_per_champion_per_class.tags[1].alias("tag2") , items_per_champion_per_class.win , items_per_champion_per_class.Info )
x = "stack(2, tag1  , tag2  ) as ( tag  )"
Stacked_items_per_class = items_per_class.select("item", expr(x) , "win")


In [7]:
All  = ['Boots of Speed',
 'Faerie Charm',
 'Rejuvenation Bead',
 "Giant's Belt",
 'Cloak of Agility',
 'Blasting Wand',
 'Sapphire Crystal',
 'Ruby Crystal',
 'Cloth Armor',
 'Chain Vest',
 'Null-Magic Mantle',
 'Long Sword',
 'Pickaxe',
 'B. F. Sword',
 "Hunter's Talisman",
 "Hunter's Machete",
 'Dagger',
 'Recurve Bow',
 'Amplifying Tome',
 'Vampiric Scepter',
 "Doran's Shield",
 "Doran's Blade",
 "Doran's Ring",
 'Negatron Cloak',
 'Needlessly Large Rod',
 'Dark Seal',
 'Cull',
 'Enchantment: Warrior',
 'Enchantment: Cinderhulk',
 'Enchantment: Runic Echoes',
 'Enchantment: Warrior',
 'Enchantment: Cinderhulk',
 'Enchantment: Runic Echoes',
 'Enchantment: Bloodrazor',
 'Enchantment: Bloodrazor',
 'Health Potion',
 'Showdown Health Potion',
 'Total Biscuit of Rejuvenation',
 'Total Biscuit of Everlasting Will',
 'Kircheis Shard',
 'Refillable Potion',
 'Corrupting Potion',
 "Oracle's Extract",
 "Guardian's Horn",
 'Poro-Snax',
 'Diet Poro-Snax',
 'Control Ward',
 "Shurelya's Reverie",
 'Elixir of Iron',
 'Elixir of Sorcery',
 'Elixir of Wrath',
 'Minion Dematerializer',
 'Commencing Stopwatch',
 'Stopwatch',
# 'Broken Stopwatch',
 'Slightly Magical Boots',
 'Perfectly Timed Stopwatch',
 #'Broken Stopwatch',
 'Abyssal Mask',
 "Archangel's Staff",
 'Manamune',
 "Berserker's Greaves",
 "Archangel's Staff (Quick Charge)",
 'Manamune (Quick Charge)',
 'Boots of Swiftness',
 'Catalyst of Aeons',
 "Sorcerer's Shoes",
 'Frozen Mallet',
 'Glacial Shroud',
 'Iceborn Gauntlet',
 'Guardian Angel',
 'Rod of Ages',
 'Chalice of Harmony',
 'Rod of Ages (Quick Charge)',
 'Hextech GLP-800',
 'Infinity Edge',
 'Mortal Reminder',
 'Last Whisper',
 "Lord Dominik's Regards",
 "Seraph's Embrace",
 "Mejai's Soulstealer",
 'Muramana',
 'Muramana',
 'Phage',
 'Phantom Dancer',
 'Ninja Tabi',
 "Seraph's Embrace",
 "Zeke's Convergence",
 "Jaurim's Fist",
 "Sterak's Gage",
 'Sheen',
 'Spirit Visage',
 'Kindlegem',
 'Sunfire Cape',
 'Tear of the Goddess',
 'Black Cleaver',
 'Bloodthirster',
 'Tear of the Goddess (Quick Charge)',
 'Ravenous Hydra',
 'Thornmail',
 'Bramble Vest',
 'Tiamat',
 'Trinity Force',
 "Warden's Mail",
 "Warmog's Armor",
 "Overlord's Bloodmail",
 "Runaan's Hurricane",
 'Zeal',
 'Statikk Shiv',
 "Rabadon's Deathcap",
 "Wit's End",
 'Rapid Firecannon',
 'Stormrazor',
 'Lich Bane',
 'Stinger',
 "Banshee's Veil",
 'Aegis of the Legion',
 'Redemption',
 'Fiendish Codex',
 "Knight's Vow",
 'Frozen Heart',
 "Mercury's Treads",
 "Guardian's Orb",
 'Aether Wisp',
 'Forbidden Idol',
 "Nashor's Tooth",
 "Rylai's Crystal Scepter",
 'Boots of Mobility',
 "Executioner's Calling",
 "Guinsoo's Rageblade",
 "Caulfield's Warhammer",
 'Serrated Dirk',
 'Void Staff',
 'Haunting Guise',
 'Dervish Blade',
 'Mercurial Scimitar',
 'Quicksilver Sash',
 "Youmuu's Ghostblade",
 "Randuin's Omen",
 'Bilgewater Cutlass',
 'Hextech Revolver',
 'Hextech Gunblade',
 'Duskblade of Draktharr',
 "Liandry's Torment",
 'Hextech Protobelt-01',
 'Blade of the Ruined King',
 'Hexdrinker',
 'Maw of Malmortius',
 "Zhonya's Hourglass",
 'Ionian Boots of Lucidity',
 'Spear of Shojin',
 'Morellonomicon',
 "Athene's Unholy Grail",
 "Head of Kha'Zix",
 'Umbral Glaive',
 'Sanguine Blade',
 "Guardian's Hammer",
 'Locket of the Iron Solari',
 "Seeker's Armguard",
 'Gargoyle Stoneplate',
 'Adaptive Helm',
 'Hex Core mk-1',
 'Hex Core mk-2',
 'Perfect Hex Core',
 #'Prototype Hex Core',
 "Spectre's Cowl",
 "Mikael's Crucible",
 "Luden's Echo",
 'Scarecrow Effigy (Trinket)',
 'Warding Totem (Trinket)',
 'Arcane Sweeper',
 'Greater Stealth Totem (Trinket)',
 'Greater Vision Totem (Trinket)',
 'Farsight Alteration',
 'Oracle Lens',
 'Molten Edge',
 'Forgefire Cape',
 "Rabadon's Deathcrown",
 'Infernal Mask',
 'Obsidian Cleaver',
 'Salvation',
 'Circlet of the Iron Solari',
 'Trinity Fusion',
 "Zhonya's Paradox",
 'Frozen Fist',
 "Youmuu's Wraithblade",
 'Might of the Ruined King',
 "Luden's Pulse",
 "'Your Cut'",
 "Head of Kha'Zix",
 "Head of Kha'Zix",
 "Head of Kha'Zix",
 "Head of Kha'Zix",
 'Ardent Censer',
 'Essence Reaver',
 'Eye of the Herald',
 'Eye of the Herald',
 'Ghost Poro',
 'Black Spear',
 'Black Spear',
 'Enchantment: Warrior',
 'Enchantment: Cinderhulk',
 'Enchantment: Runic Echoes',
 'Enchantment: Bloodrazor',
 'Frosted Snax',
 'Super Spicy Snax',
 'Espresso Snax',
 'Rainbow Snax Party Pack!',
 'Dawnbringer Snax',
 'Nightbringer Snax',
 'Cosmic Shackle',
 'Singularity Lantern',
 'Dark Matter Scythe',
 'Gravity Boots',
 'Cloak of Stars',
 'Dark Star Sigil',
 "Stalker's Blade",
 "Skirmisher's Sabre",
 "Dead Man's Plate",
 'Titanic Hydra',
 "Bami's Cinder",
 'Righteous Glory',
 'Crystalline Bracer',
 'Lost Chapter',
 "Death's Dance",
 'Edge of Night',
 "Spellthief's Edge",
 'Frostfang',
 'Shard of True Ice',
 'Steel Shoulderguards',
 'Runesteel Spaulders',
 'Pauldrons of Whiterock',
 'Relic Shield',
 "Targon's Buckler",
 'Bulwark of the Mountain',
 'Spectral Sickle',
 'Harrowing Crescent',
 'Black Mist Scythe',
 'Fire at Will',
 "Death's Daughter",
 'Raise Morale',
 'Twin Shadows',
 'Spellbinder',
 'Oblivion Orb']
basic = ["Cull" , "Doran's Blade" , "Emberknife" , "Guardian's Blade" , "Doran's Ring"  , "Doran's Shield" , "Guardian's Hammer", "Guardian's Horn",
"Guardian's Orb", "Hailblade"]
boot = ["Berserker's Greaves" , "Boots"  ,"Boots of Swiftness" , "Ionian Boots of Lucidity" , "Mercury's Treads" , "Mobility Boots" ,
        "Plated Steelcaps" , "Sorcerer's Shoes"]


epic = ["Aegis of the Legion " ,
"Aether Wisp" ,
"Bami's Cinder " ,
"Bandleglass Mirror " ,
"Blighting Jewel ",
"Bramble Vest" , 
"Caulfield's Warhammer " ,
"Chain Vest " ,
"Crystalline Bracer" ,
"Executioner's Calling" , 
"Fiendish Codex ",
"Forbidden Idol " ,
"Frostfang " ,
"Giant's Belt" ,
"Glacial Buckler " ,
"Harrowing Crescent " ,
"Hearthbound Axe" ,
"Hexdrinker" ,
"Hextech Alternator" ,
"Ironspike Whip",
"Kindlegem ",
"Kircheis Shard " ,
"Last Whisper " ,
"Leeching Leer " , 
"Lost Chapter " ,
"Negatron Cloak " ,
"Noonquiver " ,
"Oblivion Orb " ,
"Phage" ,
"Quicksilver Sash " ,
"Rageknife " ,
"Recurve Bow"  ,     
"Runesteel Spaulders ",
"Seeker's Armguard " ,
"Serrated Dirk" , 
"Sheen" , 
"Spectre's Cowl" ,
"Stirring Wardstone " ,
"Targon's Buckler " ,
"Tiamat " , 
"Vampiric Scepter " ,
"Verdant Barrier" ,
"Warden's Mail " ,
"Watchful Wardstone ",
"Winged Moonplate ","Zeal"]

lEGENDARY_MITHIC = set(All) - set(basic) -set(epic) -set(boot)



In [23]:
################################# Pick times ####################################

Item_picking_times = Stacked_items_per_cahmpion.rdd.map(lambda row: (row['item']  ,1)).reduceByKey(lambda a,b: a+b)

################################# Win rate ######################################

Winner_items =  Stacked_items_per_cahmpion.filter(col("win") == True)
Winner_winning_times = Winner_items.rdd.map(lambda row: (row['item'] ,1)).reduceByKey(lambda a,b: a+b)
Winning = Winner_winning_times.join(Item_picking_times)
items_winning_rate = Winning.map(lambda row : (row[0],row[1][0]/row[1][1]) )


################################ Synergies  & Duos #############################

Synergies_champion_item_times = Stacked_items_per_cahmpion.rdd.map(lambda row: ((row["championId"], row['item'] ) ,1)).reduceByKey(lambda a,b: a+b)
Synergies_class_item_times = Stacked_items_per_class.rdd.map(lambda row: ((row["tag"], row['item'] ) ,1)).reduceByKey(lambda a,b: a+b)

#### Creative part #1 Synergies_winning_rate
Synergies_champion_item_winning_times = Winner_items.rdd.map(lambda row: ((row["championId"], row['item'] )  ,1)).reduceByKey(lambda a,b: a+b) 
Synergies_winning = Synergies_champion_item_winning_times.join(Synergies_champion_item_times)
Synergies_winning_rate = Synergies_winning.map(lambda row : (row[0][0] , row[0][1], row[1][0]/row[1][1] ) )

################################ Item Suggestion  ###################################

boot = ["Berserker's Greaves" , "Boots"  ,"Boots of Swiftness" , "Ionian Boots of Lucidity" , "Mercury's Treads" , "Mobility Boots" ,
        "Plated Steelcaps" , "Sorcerer's Shoes"]

##### boot suggestion ###########
champion_boot_all = spark.createDataFrame(Synergies_winning_rate.filter(lambda x : x[1] in boot ) , schema = ["championId" , "item" , "win_rate"])

w = Window.partitionBy('championId')
champion_boot_best = champion_boot_all.withColumn('maxB', f.max('win_rate').over(w))\
    .where(f.col('win_rate') == f.col('maxB'))\
    .drop('maxB')
boot_suggestions = champion_boot_best.rdd.map(lambda x : (x[0] , x[1])).reduceByKey(lambda a, b: a)
boot_df = spark.createDataFrame(boot_suggestions , schema = ["dist" , "boot"])

##### basic suggestion #########
champion_basic_all = spark.createDataFrame(Synergies_winning_rate.filter(lambda x : x[1] in basic ) , schema = ["championId" , "item" , "win_rate"])
w = Window.partitionBy('championId')
champion_basic_best = champion_basic_all.withColumn('maxB', f.max('win_rate').over(w))\
    .where(f.col('win_rate') == f.col('maxB'))\
    .drop('maxB')
basic_suggestions = champion_basic_best.rdd.map(lambda x : (x[0] , x[1])).reduceByKey(lambda a, b: a)
basic_df = spark.createDataFrame(basic_suggestions , schema = ["championId" , "basic"])

##### epic suggestion ##########
champion_epic_all = spark.createDataFrame(Synergies_winning_rate.filter(lambda x : x[1] in epic ) , schema = ["championId" , "item" , "win_rate"])
w = Window.partitionBy('championId')
champion_epic_best = champion_epic_all.withColumn('maxB', f.max('win_rate').over(w))\
    .where(f.col('win_rate') == f.col('maxB'))\
    .drop('maxB')
epic_suggestions = champion_epic_best.rdd.map(lambda x : (x[0] , x[1])).reduceByKey(lambda a, b: a)
# epic_suggestions.collect()
epic_df = spark.createDataFrame(epic_suggestions , schema = ["championId" , "epic"])


#### legendary mythic suggestion #
champion_LM_all = spark.createDataFrame(Synergies_winning_rate.filter(lambda x : x[1] in lEGENDARY_MITHIC ) , schema = ["championId" , "item" , "win_rate"])
w = Window.partitionBy('championId')
champion_LM_best = champion_LM_all.withColumn('maxB', f.max('win_rate').over(w))\
    .where(f.col('win_rate') == f.col('maxB'))\
    .drop('maxB')
ML_suggestions = champion_LM_best.rdd.map(lambda x : (x[0] , x[1])).reduceByKey(lambda a, b: a)
ML_df =spark.createDataFrame(ML_suggestions , schema = ["DIST_CH" , "Mithyic or Legendary"])

#### Integration ##########

integration1 = boot_df.join(basic_df , boot_df.dist == basic_df.championId ).drop("championid" ).withColumnRenamed("dist" ,"championid" )
integration2 = epic_df.join(ML_df , epic_df.championId==ML_df.DIST_CH).drop("championid" )
integration = integration1.join(integration2 ,integration1.championid==integration2.DIST_CH).drop("DIST_CH")


In [20]:
#### Additional ###############
##### General suggestion ###### 
synergies_df_all = spark.createDataFrame(Synergies_winning_rate , schema = ["championId" , "item" , "win_rate"])

window = Window.partitionBy(synergies_df_all['championId']).orderBy(synergies_df_all['win_rate'].desc())
suggested_items_df= synergies_df_all.select('*', rank().over(window).alias('rank')).filter(col('rank') <= 5).drop("rank")

item_suggestions= suggested_items_df.rdd.map(lambda x : (x[0] , [x[1]])).reduceByKey(lambda a, b: a+b )
item_sugg = spark.createDataFrame(item_suggestions , schema=["champion" , "Items"])


In [24]:
spark.stop()